<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_ATH_TTM_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2023-11-14 16:18:04


In [2]:
!pip install yfinance

In [3]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

mypf = mypf[mypf['LatestQtr'] == 0]
#mypf = mypf[mypf['v80'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['ACC','AMBUJACEM','CADILAHC','PGHH','SIEMENS', 'GILLETTE', 'NESTLEIND', 'SANOFI', 'ADANITRANS', 'AMARAJABAT', 'SCHAEFFLER ']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100[0:80]
#stock_ids = ['DIVISLAB', 'PREMIERPOL']
curr_qtr = 'Sep 2023'
curr_year = 'Mar 2023'
print('Stocks to be scanned:',len(stock_ids))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Stocks to be scanned: 36


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_latest_analysis = pd.DataFrame()

for stock_name in stock_ids:

  print(stock_name)

  link_blueprint = "https://www.screener.in/company/"

  stock_list = ['3MINDIA', 'AUBANK', 'COLPAL', 'GLAND', 'LUXIND', 'WHIRLPOOL' , 'IEX', 'ROSSARI', 'VALIANTORG', 'VINATIORGA', 'TATAELXSI',
                'AMARAJABAT', 'EQUITAS', 'AAVAS']

  if stock_name in stock_list:
      actual_link = link_blueprint + stock_name.upper()
  else:
      actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"

  def get_list(actual_link):
      page = requests.get(actual_link)
      link_soup = BeautifulSoup(page.content, 'html.parser')

      test_list = link_soup.find(id="profit-loss").text.replace("  ", "").split('\n')

      while("" in test_list) :
          test_list.remove("")
      while("\r" in test_list) :
          test_list.remove("\r")
      while(" \r" in test_list) :
          test_list.remove(" \r")

      return test_list
  #print(actual_link)
  test_list = get_list(actual_link)

  def get_yoy(test_list):
    #print(len(test_list))
    n = len(test_list)

    if n < 100:
        actual_link = link_blueprint + stock_name.upper()
        test_list = get_list(actual_link)
        #print(len(test_list))

    #print(test_list)
    if 'TTM' not in test_list:
        end_year = curr_year
    else:
        end_year = 'TTM'

    df_yoy = pd.DataFrame()

    #print(test_list.index('Sep 2023'))
    s = test_list.index(end_year) +1 - test_list.index('Mar 2017')

    df_yoy['Year'] = test_list[test_list.index('Mar 2017'):test_list.index(end_year)+1]
    df_yoy['Sales'] = test_list[test_list.index('Expenses\xa0+')-s:test_list.index('Expenses\xa0+')]
    df_yoy['Profit'] = test_list[test_list.index('Tax %')-s:test_list.index('Tax %')]

    df_yoy.set_index('Year', inplace=True)
    df_yoy = df_yoy.T
    df_yoy['Stock'] = stock_name.upper() + '.NS'
    #df_yoy
    return df_yoy
  page = requests.get(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')
  scan_list = link_soup.find(id="quarters").text.replace("  ", "").split('\n')
  #print(scan_list.index('Sep 2023'))
  try:
    scan_list.index(curr_qtr) > 30
    df_latest_analysis = pd.concat([df_latest_analysis, get_yoy(test_list)])
  except ValueError:
      print(stock_name + ' = FAILED =')
      pass  # do nothing!

ABBOTINDIA
ABBOTINDIA = FAILED =
ALKYLAMINE
ALKYLAMINE = FAILED =
APEX
APEX = FAILED =
ASTRAZEN
ASTRAZEN = FAILED =
BANDHANBNK
BANDHANBNK = FAILED =
BAYERCROP
BAYERCROP = FAILED =
BIRLAMONEY
BIRLAMONEY = FAILED =
CAMPUS
CAMPUS = FAILED =
CAMS
CAMS = FAILED =
CLEAN
CLEAN = FAILED =
COALINDIA
CROMPTON
CROMPTON = FAILED =
CUB
CUB = FAILED =
DREAMFOLKS
DREAMFOLKS = FAILED =
EQUITAS
EQUITAS = FAILED =
EQUITASBNK
EQUITASBNK = FAILED =
HONAUT
HONAUT = FAILED =
IBULHSGFIN
IBULHSGFIN = FAILED =
ICICIGI
ICICIGI = FAILED =
IRCTC
IRCTC = FAILED =
ITBEES
ITBEES = FAILED =
JCHAC
JCHAC = FAILED =
JIOFIN
JIOFIN = FAILED =
LICI
LICI = FAILED =
NMDC
NMDC = FAILED =
PAGEIND
PAGEIND = FAILED =
PFIZER
PFIZER = FAILED =
RAJESHEXPO
RAJESHEXPO = FAILED =
RELAXO
RELAXO = FAILED =
SBICARD
SBICARD = FAILED =
SBILIFE
SBILIFE = FAILED =
STARHEALTH
STARHEALTH = FAILED =
TASTYBITE
TASTYBITE = FAILED =
TIPSINDLTD
TIPSINDLTD = FAILED =
UJJIVANSFB
UJJIVANSFB = FAILED =
WSTCSTPAPR
WSTCSTPAPR = FAILED =


In [5]:
print('Stocks updated:', df_latest_analysis.shape[0]/2)
if df_latest_analysis.shape[0] > 0:
  lastestq_stocks = list(df_latest_analysis['Stock'].unique())
  len(lastestq_stocks)

Stocks updated: 1.0


In [6]:
df_latest_analysis

Year,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,TTM,Stock
Sales,"78,164","85,244","99,586","96,080","90,026","109,715","138,252","142,081",COALINDIA.NS
Profit,"14,446","10,770","27,127","24,071","18,009","23,616","38,001","37,798",COALINDIA.NS


In [7]:
if df_latest_analysis.shape[0] > 0:
  #df_analysis.columns
  df_latest_analysis = df_latest_analysis.reset_index()
  df_latest_analysis.rename({'TTM': 'Latest', 'index': 'Type'}, axis=1, inplace=True)
  cols = [ 'Mar 2017','Mar 2018', 'Mar 2019', 'Mar 2020', 'Mar 2021', 'Mar 2022', 'Mar 2023', 'Latest']
  #df_analysis[cols] = df_analysis[cols].apply(pd.to_numeric)
  for col in cols:
      df_latest_analysis[col] = df_latest_analysis[col].str.replace(',', '').astype(float)

In [8]:
df_latest_analysis

Year,Type,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Latest,Stock
0,Sales,78164.0,85244.0,99586.0,96080.0,90026.0,109715.0,138252.0,142081.0,COALINDIA.NS
1,Profit,14446.0,10770.0,27127.0,24071.0,18009.0,23616.0,38001.0,37798.0,COALINDIA.NS


In [9]:
if df_latest_analysis.shape[0] > 0:
  df_latest_analysis['Max'] = df_latest_analysis.max(axis=1)
  df_latest_analysis['Diff%'] = round((df_latest_analysis['Max'] - df_latest_analysis['Latest'])*100/df_latest_analysis['Max'],2)

<ipython-input-9-ff4f08accfa3>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_latest_analysis['Max'] = df_latest_analysis.max(axis=1)


In [10]:
df_latest_analysis

Year,Type,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Latest,Stock,Max,Diff%
0,Sales,78164.0,85244.0,99586.0,96080.0,90026.0,109715.0,138252.0,142081.0,COALINDIA.NS,142081.0,0.00
1,Profit,14446.0,10770.0,27127.0,24071.0,18009.0,23616.0,38001.0,37798.0,COALINDIA.NS,38001.0,0.53


In [11]:
if df_latest_analysis.shape[0] > 0:
  df_master_analysis = pd.read_csv('/content/drive/My Drive/data/stocks/myProspects-ATH-TTM.csv')
  df_master_analysis.columns = df_latest_analysis.columns
  df_master_analysis = df_master_analysis[~df_master_analysis['Stock'].isin(lastestq_stocks)]
  df_master_analysis = pd.concat([df_master_analysis, df_latest_analysis])
  df_master_analysis.to_csv('/content/drive/My Drive/data/stocks/myProspects-ATH-TTM.csv', index=False)

In [12]:
if df_latest_analysis.shape[0] > 0:
  df_tmp = df_latest_analysis[df_latest_analysis['Diff%'] <= 5]
  tmp = pd.DataFrame(df_tmp['Stock'].value_counts())
  len(tmp[tmp['Stock']>1].index)
  star_stocks = list(tmp[tmp['Stock']>1].index)
  print(len(star_stocks))

1


In [13]:
mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')
mypf.loc[mypf['Stock'].isin(lastestq_stocks), 'LatestQtr'] = 1
mypf.loc[mypf['Stock'].isin(lastestq_stocks), 'StarStock'] = 0
mypf.loc[mypf['Stock'].isin(star_stocks), 'StarStock'] = 1
mypf.to_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv', index=False)